## Assignment 2 - Solutions
##### Submitted By : Anit Gupta
##### Email ID : agupta33@student.gsu.edu

### Question 1 : From the Users.xml file, find all users which are from Georgia and output to screen their DisplayName only. (20 points)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

#Creating Spark Session 
spark = SparkSession.builder.master("local[*]").appName("Learning_Spark").getOrCreate()
sc = spark.sparkContext

#Reading Users.xml
Users = sc.textFile('Users.xml');



* Here method read_value_from_xml() is used for parsing the values form xml file and it maps it into the RDD.
* It uses Regular Expression that returns any value that comes after that tag passed along with the function call.

In [2]:
import re;
def read_value_from_xml(Input, tag, cast_type=str):
    patre = r'{col}="([^"]*)"'
    pat = patre.format(col=tag)
    result = re.search(pat, Input)
    return cast_type(result[1]) if result is not None else None


* In the below cell we are checking all the location with GA/Georgia in all the cases
* We have ensure the use of "in" operator instead of "=" to get all the results such as ( Athens, GA)

Output is like :
['Tony Boyles',
 'pkerl',
 'Nick Larsen',
 ...
  'Tiji Mathew',
 'user3776637'
 ]

In [3]:
#to make sure we are only dealing with rows
onlyRows = Users.filter(lambda line: line.find('<row') >= 0)

#Method to fetch and filter the RDD for Users in Georgia.
def checkGeorgia(lst):
  location=read_value_from_xml(lst,'Location')
  name=read_value_from_xml(lst,'DisplayName')
  y= lambda a: name if(location!=None and location!="" and ("GA" in str(location) or "GEORGIA" in str(location).upper())) else None
  return y(name)

#mapping only those values which have "Georgia/GA/georgia/ga"
finalNameList = onlyRows.map(lambda line :checkGeorgia(line)).filter(lambda x: x is not None)

finalNameList.collect()


['Tony Boyles',
 'pkerl',
 'Nick Larsen',
 'gfritz',
 'Aleksandr Blekh',
 'Michael',
 'Ayush',
 'azoorob',
 'ontek',
 'Aravind R. Yarram',
 'ilya',
 'vkb',
 'Daisuke Aramaki',
 'tempusfugit',
 'Henry Crutcher',
 'Goddard',
 'Matt Simpson',
 'Peter Woolfitt',
 'Giorgi Gzirishvili',
 'matt biskup',
 'Jason W',
 'Peter Mourfield',
 'Magsol',
 'Bob Baxley',
 'Alex Azazel',
 'badjr',
 'mplunney',
 'YC Hu',
 'ryan',
 'Patrick Gerbes',
 'Ilya Lapitan',
 'Dan Anton',
 'pradyumnad',
 'Manish Ranjan',
 'Dato Janez',
 'Psidom',
 'Teresa Madsen',
 'Neuromeda',
 'Brandon',
 'jpm',
 'Mr. Rooter of Savannah',
 'Mr. Rooter of Southeast GA',
 'Abesalomi Gogatishvili',
 'Khiem Ha',
 'Jenna Kwon',
 'Tedo G.',
 'mlibre',
 'Ahmet Cecen',
 'Jeff',
 'Guy Gordon',
 'C3Theo',
 'niru dyogi',
 'Vinitha Palani',
 'Mac18',
 'Andrew',
 'Aditya Gogoi',
 'turtlemonvh',
 'Lewis Rodgers',
 'Tarun Luthra',
 'Jesse Scherer',
 'Devendra Lattu',
 'cosmosa',
 'Todd Dawson',
 'ironv',
 'Mboolean',
 'Jimd',
 'Melinda Weathers

### Question 2 :Using the Users.xml file, provide the count for all users which joined (CreationDate) in 2017. (30 points). Output this to the screen.

* Here we are couting every date which has 2017 in the file.
* output is : 14239 people who joined in year 2017

In [4]:
#Using a function to read the xml and get all the dates which are in 2017
def checkCreadtionDate(lst):
  date=read_value_from_xml(lst,'CreationDate')
  y= lambda a: date if ("2017" in date and date!="") else None
  return y(date)

#Data Preprocessing 
interestedDatesOnly = onlyRows.map(lambda line :checkCreadtionDate(line)).filter(lambda x: x is not None )

print("The count for all users which joined (CreationDate) in 2017 are below : ")
interestedDatesOnly.count()

The count for all users which joined (CreationDate) in 2017 are below : 


14239

### Question 3 : Using the PostHistory file, count the number of Posts that feature the words “Spark” and “Scala”. Output this to the screen. (20 points)


* We now read file PostHistory to work further.
* We count all the posts which have both SCALA and SPARK in them in all the cases.

The out put will be 211 posts that have both spark and scala

In [5]:
#Reading the PostHistory file and filtering out all the "non row values" from the XML"
PHFile = sc.textFile('PostHistory.xml');
PHFileData = PHFile.filter(lambda line: line.find('<row') >= 0)

In [6]:
#Using function to read the XML and get all the texts and then returing only those texts which have "Spark and Scala" in it 
def checkScalaSparkPosts(Data):
  Data=read_value_from_xml(Data,'Text')
  y= lambda a: True if (("SCALA" in str(Data).upper() and Data!="") and ("SPARK" in str(Data).upper() and Data!="")) else None
  return y(Data)

#Using function to read the XML and get all the texts and then returing only those texts which have "Spark and Scala" 
#in any case.

PostCount = PHFileData.map(lambda line :checkScalaSparkPosts(line)).filter(lambda x: x is not None )

print("Number of posts that contain word SCALA and SPARK in it are: ")
PostCount.count()

Number of posts that contain word SCALA and SPARK in it are: 


211

### Question 4: Using the PostHistory file, provide a total count of the words used by each distinct user. In other words, count all words in all posts for each user and display this to screen. You can only identify users by the UserID (30 points). You get 15 bonus points if you get the actual DisplayName of the user.

Here we tweak our function a little bit and return 0 to make our calculations easy

In [7]:
import re;
def read_value_from_xml_ret0(Input, tag, cast_type=str):
    patre = r'{col}="([^"]*)"'
    pat = patre.format(col=tag)
    result = re.search(pat, Input)
    return cast_type(result[1]) if result is not None else 0


In [8]:
#FINAL ANSWER 4
from collections import Counter
from operator import add

#reading both the files
Users = sc.textFile('Users.xml');
PHFile = sc.textFile('PostHistory.xml');

#Filtering only row data from both the files.
Users = Users.filter(lambda line: line.find('<row') >= 0)
PHFileData = PHFile.filter(lambda line: line.find('<row') >= 0)

#Getting DisplayName and the UserID from the XML
DispNameRDD=Users.map(lambda a: ((read_value_from_xml(a,"Id"),(str(read_value_from_xml(a,"DisplayName"))))))

#Getting all the Text values for all the users. then splitting it into words and the counting them.
UserTextRDD=PHFileData.map(lambda a: ((read_value_from_xml_ret0(a,"UserId"),len(str(read_value_from_xml_ret0(a,"Text")).split()))))

#combining all the tuples on the basis of the user to find the total words written by each user.
UserTextCountRDD=UserTextRDD.reduceByKey(add)

#joining the display name with the UserID
allRDD=DispNameRDD.join(UserTextCountRDD)
   
#Formatting  the RDD so that it can be readble now
ReadableDataRDD=allRDD.map(lambda vie:(("Name: "+str((vie[1])[0])),"UserID: "+str(vie[0]),"Word Count: "+str((vie[1])[1])))

#printitng the RDD
ReadableDataRDD.collect()


[('Name: Doorknob', 'UserID: 5', 'Word Count: 96'),
 ('Name: Robert Cartaino', 'UserID: 50', 'Word Count: 6'),
 ('Name: Shog9', 'UserID: 61', 'Word Count: 460'),
 ('Name: Igor Bobriakov', 'UserID: 88', 'Word Count: 361'),
 ('Name: EdChum', 'UserID: 95', 'Word Count: 1186'),
 ('Name: Johnny000', 'UserID: 115', 'Word Count: 2264'),
 ('Name: Konstantin V. Salikhov', 'UserID: 118', 'Word Count: 717'),
 ('Name: Rhand', 'UserID: 127', 'Word Count: 186'),
 ('Name: joker', 'UserID: 131', 'Word Count: 320'),
 ('Name: DVK', 'UserID: 132', 'Word Count: 840'),
 ('Name: akellyirl', 'UserID: 136', 'Word Count: 258'),
 ('Name: Marco13', 'UserID: 156', 'Word Count: 3040'),
 ('Name: Wojciech Walczak', 'UserID: 173', 'Word Count: 3212'),
 ('Name: samthebrand', 'UserID: 177', 'Word Count: 186'),
 ('Name: Filipe Ferminiano', 'UserID: 199', 'Word Count: 308'),
 ('Name: Tahir Akhtar', 'UserID: 200', 'Word Count: 205'),
 ('Name: sushant-hiray', 'UserID: 226', 'Word Count: 183'),
 ('Name: organic agave', 'Use

### Question 5 : GRADUATE STUDENTS: Using the users.xml, comments.xml and PostHistory.xml files, produce a single file that includes the following information: DisplayName, Number of Comments, total Score and Number of posts. This file should have the users (DisplayName) sorted by score, descending from higher to lower. (40 points)

We did this question in the following steps:
* First we are reading from all the files teh necessary columns
* Then in order to get them all in one file we are performing join so that the data comes for all the users
* After that we are preprocessing  the data such that we remove the None Values with 0.
* Then we are sorting the file on the basis of the Score.
* Then we are using the package "Pretty Table" to make our data look pretty and nice in the File
* After that we put the data in the file. we made sure its single file output only.


And the output looks like below :<br>
('ID :381',<br>
      'Name :Emre',<br>
         'Total Score :684',<br>
            'Number of Posts :440',<br>
                  'Number of Comments :1246')<br>
      


In [9]:
import re;
def read_value_from_xml_ret2(Input, tag, cast_type=str):
    patre = r'{col}="([^"]*)"'
    pat = patre.format(col=tag)
    result = re.search(pat, Input)
    return 1 if result is not None else 0

In [13]:
from operator import add
from prettytable import PrettyTable

PostHistoryFile = sc.textFile('PostHistory.xml');
PostHistoryFile = PostHistoryFile.filter(lambda line: line.find('<row') >= 0)

UsersData = sc.textFile('Users.xml');
UsersData = UsersData.filter(lambda line: line.find('<row') >= 0)

CommentsData = sc.textFile('Comments.xml');
CommentsData = CommentsData.filter(lambda line: line.find('<row') >= 0)

#getting UserID and Display Names in one RDD 
print("Getting DisplayName...")
DispNameRDD=Users.map(lambda a: ((read_value_from_xml(a,"Id"),(str(read_value_from_xml(a,"DisplayName"))))))

#getting UserID and Count of  comments per User in RDD
print("Calculating total Number of Comment ...")
AllCommentsCount=CommentsData.map(lambda a: ((read_value_from_xml_ret0(a,"UserId"),((read_value_from_xml_ret2(a,"Text")))))).reduceByKey(add)


#getting UserID and Total Score per User in RDD
print("Calculating total Score of the User...")
AllScoreTotal=CommentsData.map(lambda a: ((read_value_from_xml_ret0(a,"UserId"),(int(read_value_from_xml_ret0(a,"Score")))))).reduceByKey(add)


#getting UserID and Total number of posts per User
print("Calculating total Number of posts...")
AllPostCount=PostHistoryFile.map(lambda a: ((read_value_from_xml_ret0(a,"UserId"),(int(read_value_from_xml_ret2(a,"Text")))))).reduceByKey(add)


#getting all the data in one RDD using Joins
print("Joining all the data as Unstructured data ...")
UnstructuredDataRDD=DispNameRDD.leftOuterJoin(AllCommentsCount).leftOuterJoin(AllScoreTotal).leftOuterJoin(AllPostCount)

#Structuring the data into proper list of tuples.
print ("Structuring the data into proper list of tuples ...")
FinalAllDatawoCaption=UnstructuredDataRDD.map(lambda t:(str(t[0]),(((t[1])[0])[0])[0],(((t[1])[0])[1]),((t[1])[1]),((((t[1])[0])[0])[1])))


#Preprocessing the Data and removing the None Values afer joining
print("Preprocessing the Data and removing the None Values afer joining...")
ewRDD = FinalAllDatawoCaption.map(lambda x: ((0 if x[0] is None else x[0]), (0 if x[1] is None else x[1]),(0 if x[2] is None else x[2]),(0 if x[3] is None else x[3]),(0 if x[4] is None else x[4]))).sortBy(lambda x: (x[2]))

#Sorting RDD in descending format on the basis of each User.
SortedRDD=ewRDD.sortBy(lambda x: (x[2]),ascending=False)

#formatting the RDD into readable format
print("Formatting the RDD...")
FinalAllData=SortedRDD.map(lambda t:("ID :"+str(t[0]),"Name :"+t[1],"Total Score :"+str(t[2]),"Number of Posts :"+str(t[3]),"Number of Comments :"+str(t[4])))
FinalAllData.collect()

#Using pretty table since we are not about to put these to file so that it looks good
x = PrettyTable(["ID", "Name", "Total Score", "Number of Posts", "Number of Comments"])
x.align["Name"] = "l" 
x.align["ID"] = "l" 
x.padding_width = 1 

#Writtin the data to the file 
for i in FinalAllData.take(FinalAllData.count()) :
    x.add_row(i)
with open('Final_output_data.txt', 'w') as f:
    f.write(str(x))

FinalAllData.collect()

Getting DisplayName...
Calculating total Number of Comment ...
Calculating total Score of the User...
Calculating total Number of posts...
Joining all the data as Unstructured data ...
Structuring the data into proper list of tuples ...
Preprocessing the Data and removing the None Values afer joining...
Formatting the RDD...


[('ID :381',
  'Name :Emre',
  'Total Score :684',
  'Number of Posts :440',
  'Number of Comments :1246'),
 ('ID :836',
  'Name :Neil Slater',
  'Total Score :544',
  'Number of Posts :1173',
  'Number of Comments :1199'),
 ('ID :471',
  'Name :Spacedman',
  'Total Score :177',
  'Number of Posts :111',
  'Number of Comments :325'),
 ('ID :28175',
  'Name :Vaalizaadeh',
  'Total Score :170',
  'Number of Posts :1241',
  'Number of Comments :839'),
 ('ID :21',
  'Name :Sean Owen',
  'Total Score :150',
  'Number of Posts :1039',
  'Number of Comments :345'),
 ('ID :8820',
  'Name :Martin Thoma',
  'Total Score :132',
  'Number of Posts :629',
  'Number of Comments :277'),
 ('ID :35644',
  'Name :Aditya',
  'Total Score :123',
  'Number of Posts :221',
  'Number of Comments :627'),
 ('ID :924',
  'Name :Anony-Mousse',
  'Total Score :117',
  'Number of Posts :447',
  'Number of Comments :463'),
 ('ID :23305',
  'Name :oW_',
  'Total Score :101',
  'Number of Posts :421',
  'Number of Co